In [1]:
import numpy as np

In [2]:
a=np.load("input_data_emotion.npy")
b=np.load("label.np.npy")

In [3]:
a.shape

(35887, 48, 48, 3)

In [4]:
b.shape

(35887,)

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
    train_test_split(a, b, 
                     test_size=0.20,
                     stratify=b,
                     random_state=1)

In [6]:
mean_vals = np.mean(X_train, axis=0)
std_val = np.std(X_train)

In [7]:
X_train_centered = (X_train - mean_vals)/std_val
X_valid_centered = (X_test - mean_vals)/std_val

In [8]:
X_train.shape

(28709, 48, 48, 3)

In [9]:
from tensorflow.keras.utils import to_categorical

y_train_onehot = to_categorical(y_train)
y_valid_onehot = to_categorical(y_test)

In [10]:
from tensorflow.keras import layers, models

In [11]:
model = models.Sequential()

In [12]:
model.add(layers.Conv2D(32, (5, 5), padding='valid', 
                        activation='relu', input_shape=(48, 48,3)))
model.add(layers.MaxPool2D((2, 2)))
model.add(layers.Conv2D(64, (5, 5), padding='valid', 
                        activation='relu'))
model.add(layers.MaxPool2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(9*9*64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(7, activation='softmax'))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 44, 44, 32)        2432      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 22, 22, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 18, 18, 64)        51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 9, 9, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 5184)              0         
_________________________________________________________________
dense (Dense)                (None, 5184)              26879040  
_________________________________________________________________
dropout (Dropout)            (None, 5184)              0

In [14]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['acc'])

In [15]:
import time
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

callback_list = [ModelCheckpoint(filepath='cnn_checkpoint.h5', 
                                 monitor='val_loss', 
                                 save_best_only=True), 
                 TensorBoard(log_dir="logs/{}".format(time.asctime()))]

In [16]:
history = model.fit(X_train, y_train_onehot, 
                    batch_size=2048, epochs=100, 
                    validation_data=(X_test, y_valid_onehot),
                    callbacks=callback_list)

Epoch 1/100
15/15 [==============================] - 3s 88ms/step - loss: 222.4333 - acc: 0.1816 - val_loss: 1.9313 - val_acc: 0.2509
Epoch 2/100
15/15 [==============================] - 1s 64ms/step - loss: 1.9197 - acc: 0.2494 - val_loss: 1.8811 - val_acc: 0.2503
Epoch 3/100
15/15 [==============================] - 1s 64ms/step - loss: 1.8679 - acc: 0.2492 - val_loss: 1.8364 - val_acc: 0.2506
Epoch 4/100
15/15 [==============================] - 1s 64ms/step - loss: 1.8283 - acc: 0.2498 - val_loss: 1.8186 - val_acc: 0.2513
Epoch 5/100
15/15 [==============================] - 1s 64ms/step - loss: 1.8109 - acc: 0.2555 - val_loss: 1.8136 - val_acc: 0.2510
Epoch 6/100
15/15 [==============================] - 1s 64ms/step - loss: 1.8086 - acc: 0.2530 - val_loss: 1.8168 - val_acc: 0.2519
Epoch 7/100
15/15 [==============================] - 1s 64ms/step - loss: 1.8035 - acc: 0.2574 - val_loss: 1.8134 - val_acc: 0.2517
Epoch 8/100
15/15 [==============================] - 1s 64ms/step - loss: 

15/15 [==============================] - 1s 65ms/step - loss: 1.7581 - acc: 0.2791 - val_loss: 1.9229 - val_acc: 0.2570
Epoch 63/100
15/15 [==============================] - 1s 65ms/step - loss: 1.7590 - acc: 0.2770 - val_loss: 1.9293 - val_acc: 0.2548
Epoch 64/100
15/15 [==============================] - 1s 65ms/step - loss: 1.7486 - acc: 0.2806 - val_loss: 1.9695 - val_acc: 0.2593
Epoch 65/100
15/15 [==============================] - 1s 65ms/step - loss: 1.7439 - acc: 0.2815 - val_loss: 1.9582 - val_acc: 0.2593
Epoch 66/100
15/15 [==============================] - 1s 65ms/step - loss: 1.7360 - acc: 0.2846 - val_loss: 1.9646 - val_acc: 0.2605
Epoch 67/100
15/15 [==============================] - 1s 65ms/step - loss: 1.7631 - acc: 0.2760 - val_loss: 1.9326 - val_acc: 0.2586
Epoch 68/100
15/15 [==============================] - 1s 65ms/step - loss: 1.7705 - acc: 0.2720 - val_loss: 1.9549 - val_acc: 0.2565
Epoch 69/100
15/15 [==============================] - 1s 66ms/step - loss: 1.7554 

In [17]:
y_valid_onehot.shape

(7178, 7)

In [18]:
model.save('cnn_model.h5')

In [19]:
from tensorflow.keras.models import load_model

restored_model = load_model('cnn_model.h5')
restored_model.load_weights('cnn_checkpoint.h5')

In [20]:
restored_model.evaluate(X_test, y_valid_onehot)

225/225 [==============================] - 0s 1ms/step - loss: 1.8132 - acc: 0.2519


[1.8131568431854248, 0.2518807351589203]

In [21]:
pred=np.argmax(restored_model.predict(X_test), axis=1)

In [22]:
pred.shape

(7178,)

In [23]:
import tensorflow as tf
tf.math.confusion_matrix(y_test,pred)

<tf.Tensor: shape=(7, 7), dtype=int32, numpy=
array([[   0,    0,    2,  987,    1,    1,    0],
       [   0,    0,    0,  109,    0,    0,    0],
       [   0,    0,    4, 1015,    2,    2,    1],
       [   0,    0,    5, 1788,    4,    1,    0],
       [   2,    0,    1, 1224,    5,    8,    0],
       [   1,    0,    3, 1199,    3,   10,    0],
       [   0,    0,    0,  799,    0,    0,    1]], dtype=int32)>